In [80]:
import os 
import json
import pickle
import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import Normalizer

import numpy as np
import pandas as pd
from pprint import pprint
from bisect import bisect_left as find_prev
import matplotlib.pyplot as plt

import xml.etree.ElementTree as xml

strptime = datetime.datetime.strptime
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# READING 


In [5]:
#df_kept.to_csv('chosen_segments_'+str(cut)+'.csv')


cut=0.5
df_kept=pd.read_csv('chosen_segments_'+str(cut)+'.csv')
df_kept['unique_id']=df_kept['shot_id'].map(str)+'_'+df_kept['file'].map(str)
df_kept['unique_id']=df_kept['unique_id'].str.replace('.mp4','')

In [149]:
df_kept.character.unique()

array(['Peggy Mitchell', 'Tanya Branning', 'Jack Branning',
       'Archie Mitchell', 'Max Branning'], dtype=object)

In [69]:
soap_opera_scale = {'extramarital affair': 1/1.98, 'get divorced': 1/1.96,'illegitimate child': 1/1.45,'institutionalized for emotional problem': 1/1.43,'happily married': 1/4.05,'serious accident': 1/2.96,'murdered': 1/1.81,'attempt suicide': 1/1.26,'blackmailed': 1/1.86,'unfaithful spouse': 1/2.23,'sexually assaulted': 1/2.60,'abortion': 1/1.41}
life_events_scale = {'death relative': 100, 'Divorce':	73, 'Marital separation':	65, 'Imprisonment':	63, 'injury illness':	53, 'Marriage':	50, 'Fired':	47, 'Marital reconciliation':	45, 'Retirement':	45, 'Pregnancy':	40, 'Sexual difficulties':	39, 'New family member':	39, 'Business readjustment':	39, 'Money change':	38, 'Work change': 36, 'Arguing':	35, 'Mortgage':	32, 'Child leaving home':	29, 'Trouble with in-laws':	29,  'Achievement':	28}

In [178]:
# which one is what?
# original soap opera scale
soap_1_path   ='../other_experiments/Hugging_Face_classification/soap_opera_scale_Max_Jack_Tanya.csv' 
soap_2_path   ='../other_experiments/Hugging_Face_classification/soap_opera_Peggy_Archie.csv' 
# on keyword "event"
events_1_path ='../other_experiments/Hugging_Face_classification/events_ranking_Max_Jack_Tanya.csv'
# original life scale ranking (concatenate the first and second)
life_1_path   ='../other_experiments/Hugging_Face_classification/life_scale_ranking_Max_Jack_Tanya.csv'
life_end_path ='../other_experiments/Hugging_Face_classification/life_scale_ranking_end_Max_Jack_Tanya.csv'
# on keyword "daily life"
classification_results_path='../other_experiments/Hugging_Face_classification/daily_life_Max_Jack_Tanya.csv'
# life scale with improved labels (top 20)
reduced_life_clean ='../other_experiments/Hugging_Face_classification/reduced_life_scale_Max_Jack_Tanya _clean.csv'

extra_columns = ['index', 'Unnamed: 0.1', 'Unnamed: 0']


hf_results=pd.read_csv(reduced_life_clean)
hf_results=pd.read_csv(soap_1_path)
hf_results=pd.read_csv(soap_2_path)


hf_results=hf_results.drop(columns=[c for c in extra_columns if c in hf_results.columns])
hf_results['filename']=hf_results['filename'].str.replace('.xml','')
hf_results['unique_id']=hf_results['shot_id'].map(str)+'_'+hf_results['filename'].map(str)
hf_results.head()

,begin,end,filename,shot_id,transcript,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score,unique_id
0,01:40:00:10,01:40:07:09,5321170704253895214,1901,"Someone, get her out of here will you, Phil? It was there, it was there. Someone must have it. Someone must have it!",0.441644,0.356732,0.287323,0.329269,0.000222,0.317943,0.365398,0.259843,0.244835,0.420774,0.347512,0.210511,3.582005,1901_5321170704253895214
1,00:04:05:13,00:04:09:10,5300433743159556513,41,Gumbo? ..kill the baby.,0.354302,0.005846,0.664181,0.252994,0.000040,0.327603,0.539394,0.051286,0.210633,0.199810,0.221253,0.642378,3.469720,41_5300433743159556513
2,02:22:14:24,02:22:16:07,5321170704253895214,2681,"My dad made me give her up, I didn't want to... No...",0.378366,0.492466,0.673177,0.100721,0.000013,0.067842,0.104968,0.039400,0.449365,0.414013,0.214366,0.498949,3.433646,2681_5321170704253895214
3,01:09:54:11,01:09:58:05,5305617339184877785,1088,"Would you care to qualify that statement? Well, just cos Count whatsit ain't interested, she blows her brains out.",0.240351,0.426303,0.034555,0.025603,0.000007,0.326202,0.551544,0.785344,0.091333,0.314738,0.011943,0.617854,3.425776,1088_5305617339184877785
4,00:02:17:17,00:02:20:12,5300433743159556513,18,You took her with you to get rid of her grandchild.,0.470687,0.005826,0.626103,0.116069,0.000009,0.128693,0.256579,0.041866,0.465530,0.339901,0.200751,0.680625,3.332637,18_5300433743159556513


In [179]:
hf_results.describe()

,shot_id,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score
count,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000
mean,951.830040,0.081882,0.038687,0.042061,0.054760,0.010100,0.051778,0.014877,0.028587,0.053996,0.103876,0.019452,0.020734,0.520790
std,602.415155,0.089308,0.064568,0.077767,0.063066,0.027962,0.068541,0.047209,0.052582,0.092655,0.107801,0.036368,0.048967,0.469799
min,1.000000,0.000009,0.000017,0.000025,0.000038,0.000003,0.000008,0.000013,0.000013,0.000017,0.000009,0.000008,0.000016,0.000382
25%,432.000000,0.018341,0.003562,0.005525,0.012660,0.000142,0.003344,0.000579,0.002451,0.007335,0.021260,0.002219,0.001649,0.168565
50%,922.000000,0.053000,0.015201,0.016521,0.033945,0.001144,0.021239,0.002386,0.011624,0.018540,0.065815,0.007581,0.007569,0.387532
75%,1397.000000,0.113974,0.045251,0.043412,0.074036,0.006126,0.073970,0.009120,0.033700,0.050681,0.151279,0.020416,0.021778,0.738361
max,2730.000000,0.503572,0.508671,0.688707,0.660096,0.246274,0.336912,0.551544,0.785344,0.533121,0.447561,0.382601,0.707431,3.582005


In [180]:
if 'extramarital affair' in hf_results.columns:
    for c in hf_results.columns:
        if c in soap_opera_scale:
            hf_results[c] = hf_results[c] / soap_opera_scale[c]
elif 'death relative' in hf_results.columns:
    for c in hf_results.columns:
        if c in life_events_scale:
            hf_results[c] = hf_results[c] / life_events_scale[c]

In [181]:
hf_results.describe()

,shot_id,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,sexually assaulted,abortion,score
count,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000,10526.000000
mean,951.830040,0.162126,0.075826,0.060988,0.078306,0.040906,0.153262,0.026928,0.036020,0.100433,0.231644,0.050576,0.029235,0.520790
std,602.415155,0.176829,0.126553,0.112762,0.090184,0.113246,0.202883,0.085448,0.066253,0.172339,0.240396,0.094557,0.069043,0.469799
min,1.000000,0.000017,0.000033,0.000037,0.000054,0.000011,0.000025,0.000024,0.000016,0.000032,0.000021,0.000020,0.000023,0.000382
25%,432.000000,0.036316,0.006981,0.008012,0.018104,0.000575,0.009898,0.001049,0.003089,0.013642,0.047411,0.005769,0.002325,0.168565
50%,922.000000,0.104940,0.029794,0.023955,0.048542,0.004632,0.062867,0.004319,0.014647,0.034484,0.146769,0.019712,0.010672,0.387532
75%,1397.000000,0.225669,0.088692,0.062947,0.105871,0.024810,0.218950,0.016507,0.042462,0.094267,0.337352,0.053081,0.030707,0.738361
max,2730.000000,0.997073,0.996995,0.998624,0.943937,0.997410,0.997261,0.998294,0.989533,0.991604,0.998061,0.994763,0.997477,3.582005


In [182]:
# hf_results.sort_values(by=['extramarital affair'],ascending=False).head(3)

In [183]:
apply_coefficients = True
transform = np.log

cols_to_drop=['begin', 'end', 'shot_id', 'filename', 'transcript','score',
              'Marital separation','New family member','Business readjustment', 
              'Money change', 'Work change', 'Arguing','Mortgage', 'Child leaving home', 
              'Trouble with in-laws', 'Achievement', 'happily married']#

min_max_scaler  = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

chosen_cols = [c for c in hf_results.columns if c in ['death relative', 'Divorce', 'Imprisonment', 'injury illness', 'Marriage','Fired', 'Marital reconciliation', 'Retirement', 'Pregnancy','Sexual difficulties', 
                                                      'extramarital affair', 'get divorced', 'illegitimate child', 'institutionalized for emotional problem', 'serious accident', 'murdered', 'attempt suicide', 'blackmailed', 'unfaithful spouse', 'sexually assaulte', 'abortion']]

final_results = hf_results.copy()
final_results[chosen_cols] = robust_scaler.fit_transform(final_results[chosen_cols])

if apply_coefficients:
    for c in hf_results.columns:
        if 'extramarital affair' in hf_results.columns:
            if c in soap_opera_scale:
                hf_results[c] = hf_results[c] * transform(soap_opera_scale[c])
        elif 'death relative' in hf_results.columns:
            if c in life_events_scale:
                hf_results[c] = hf_results[c] * transform(life_events_scale[c])

In [184]:
final_results['score_max'] = final_results.drop([c for c in cols_to_drop if c in final_results.columns], axis=1).max(axis=1)

final_results = final_results.sort_values(by=['score_max'],ascending=False)

filtered_results = pd.merge(final_results,df_kept,on='unique_id')

filtered_result_char=filtered_results[filtered_results.character == 'Archie Mitchell']
filtered_result_char=filtered_result_char[filtered_result_char.transcript.str.split().apply(len)>5]

filtered_result_char[['transcript', 'unique_id'] + chosen_cols + ['score_max']].head(20)

"final_results['score_max'] = final_results.drop([c for c in cols_to_drop if c in final_results.columns], axis=1).max(axis=1)\n\nfinal_results = final_results.sort_values(by=['score_max'],ascending=False)\n\nfiltered_results = pd.merge(final_results,df_kept,on='unique_id')\n\nfiltered_result_char=filtered_results[filtered_results.character == 'Archie Mitchell']\nfiltered_result_char=filtered_result_char[filtered_result_char.transcript.str.split().apply(len)>5]\n\nfiltered_result_char[['transcript', 'unique_id'] + chosen_cols + ['score_max']].head(20)"

In [191]:
n_words = 10
for character in ['Peggy Mitchell', 'Archie Mitchell', 'Max Branning', 'Jack Branning', 'Tanya Branning']:
    filtered_result_char=filtered_results[filtered_results.character == character]
    # filtered_result_char=filtered_result_char[filtered_result_char.transcript.str.split().apply(len)>n_words]

    print(len(filtered_result_char), 'lines for', character)
    
    top20 = filtered_result_char[['transcript', 'filename', 'unique_id'] + chosen_cols + ['score_max']].head(40)
    top20.to_csv('final_'+character+'_nominwords.csv')

352 lines for Peggy Mitchell
328 lines for Archie Mitchell
148 lines for Max Branning
85 lines for Jack Branning
412 lines for Tanya Branning


In [157]:
final_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,10526.000000,1.052600e+04,1.052600e+04,1.052600e+04,10526.000000,10526.000000,10526.000000,1.052600e+04,10526.000000,10526.000000
mean,-0.302006,-5.633555e-01,-6.741219e-01,-3.391300e-01,-0.432402,-1.462536,-0.542832,-8.179835e-01,-0.292734,-0.654044
std,0.933856,1.548793e+00,2.052630e+00,1.027538e+00,0.970489,5.527649,1.682703,2.137546e+00,0.829120,2.432646
min,-4.711469,-1.183690e+01,-1.774213e+01,-1.020190e+01,-4.469669,-64.300185,-24.760192,-1.187129e+01,-2.936082,-34.768783
25%,-0.637587,-7.208081e-01,-7.097770e-01,-6.531947e-01,-0.746624,-0.788420,-0.706449,-7.415001e-01,-0.657317,-0.705906
50%,0.000000,-3.154774e-17,4.250073e-17,-5.526043e-17,0.000000,0.000000,0.000000,-3.467075e-17,0.000000,0.000000
75%,0.362413,2.791919e-01,2.902230e-01,3.468053e-01,0.253376,0.211580,0.293551,2.584999e-01,0.342683,0.294094
max,0.554110,3.642288e-01,4.353863e-01,5.524598e-01,0.300606,0.277874,0.371597,4.273105e-01,0.506129,0.375204


In [135]:
hf_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,0.073696,0.031984,0.007436,0.010415,0.082904,0.175562,0.008549,0.001861,0.038786,0.146258,0.002959
std,0.080813,0.053709,0.012574,0.012390,0.225259,0.234675,0.027992,0.003224,0.066889,0.151690,0.005926
min,0.000008,0.000013,0.000005,0.000007,0.000026,0.000037,0.000009,0.000001,0.000017,0.000021,0.000003
25%,0.016647,0.003192,0.001178,0.002424,0.001209,0.011431,0.000366,0.000180,0.005260,0.031100,0.000280
50%,0.047401,0.013357,0.003285,0.006470,0.009096,0.071242,0.001434,0.000803,0.013327,0.094065,0.001210
75%,0.100969,0.036775,0.008208,0.013876,0.051052,0.249845,0.005194,0.002244,0.036014,0.209391,0.003329
max,0.465219,0.451402,0.137373,0.125991,1.947709,1.175338,0.349774,0.052845,0.383295,0.640614,0.117495


In [136]:
hf_results[chosen_cols].describe()

,extramarital affair,get divorced,illegitimate child,institutionalized for emotional problem,happily married,serious accident,murdered,attempt suicide,blackmailed,unfaithful spouse,abortion
count,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000,12126.000000
mean,0.073696,0.031984,0.007436,0.010415,0.082904,0.175562,0.008549,0.001861,0.038786,0.146258,0.002959
std,0.080813,0.053709,0.012574,0.012390,0.225259,0.234675,0.027992,0.003224,0.066889,0.151690,0.005926
min,0.000008,0.000013,0.000005,0.000007,0.000026,0.000037,0.000009,0.000001,0.000017,0.000021,0.000003
25%,0.016647,0.003192,0.001178,0.002424,0.001209,0.011431,0.000366,0.000180,0.005260,0.031100,0.000280
50%,0.047401,0.013357,0.003285,0.006470,0.009096,0.071242,0.001434,0.000803,0.013327,0.094065,0.001210
75%,0.100969,0.036775,0.008208,0.013876,0.051052,0.249845,0.005194,0.002244,0.036014,0.209391,0.003329
max,0.465219,0.451402,0.137373,0.125991,1.947709,1.175338,0.349774,0.052845,0.383295,0.640614,0.117495
